In [139]:
#import keras
#from keras.datasets import cifar100
#from keras.preprocessing.image import ImageDataGenerator
#from keras.models import Sequential
#from keras.layers import Dense, Dropout, Activation, Flatten
#from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
#from keras import optimizers
import numpy as np
#from keras.layers.core import Lambda
#from keras.constraints import maxnorm
#from keras.optimizers import SGD
#from keras import backend as K
#from keras import regularizers
from keras.utils import np_utils
import os
import numpy as np
import scipy.io
import scipy.misc
import matplotlib.pyplot as plt
from matplotlib.pyplot import imread as imaread
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
import warnings

from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg16 import decode_predictions

from PIL import Image

import scipy.io as sio


from sklearn.preprocessing import LabelEncoder
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import multilabel_confusion_matrix
import matplotlib.pyplot as plt  
from sklearn.datasets import make_classification
from sklearn.metrics import plot_confusion_matrix
from sklearn.model_selection import train_test_split

In [140]:
num_classes = 102
weight_decay = 0.0005
x_shape = [32,32,3]

def imread(path):
    img = imaread(path).astype(np.float)
    if len(img.shape) == 2:
        img = np.transpose(np.array([img, img, img]), (2, 0, 1))
    return img

In [119]:
path = '/Users/utkarsh/Desktop/study/iitj/sem2/ml2/assignment2/101_ObjectCategories'
valid_exts = [".jpg", ".gif", ".png", ".jpeg"]
print ("[%d] CATEGORIES ARE IN \n %s" % (len(os.listdir(path)), path))

[10] CATEGORIES ARE IN 
 /Users/utkarsh/Desktop/study/iitj/sem2/ml2/assignment2/101_ObjectCategories


In [142]:
categories = sorted(os.listdir(path))
ncategories = len(categories)
imgs = []
labels = []
cat_dict = {}
# LOAD ALL IMAGES 
for i, category in enumerate(categories):
    iter = 0
    for f in os.listdir(path + "/" + category):
        if iter == 0:
            ext = os.path.splitext(f)[1]
            if ext.lower() not in valid_exts:
                continue
            fullpath = os.path.join(path + "/" + category, f)
            img = np.array(Image.fromarray(imread(fullpath), "RGB").resize((224, 224)))
            img = img.astype('float32')
            #img[:,:,0] -= 123.68
            #img[:,:,1] -= 116.78
            #img[:,:,2] -= 103.94
            imgs.append(img) # NORMALIZE IMAGE 
            label_curr = i
            labels.append(label_curr)
            cat_dict[category] = i
        #iter = (iter+1)%10;
print ("Num imgs: %d" % (len(imgs)))
print ("Num labels: %d" % (len(labels)) )
print (ncategories)
print(cat_dict)

Num imgs: 60
Num labels: 60
10
{'accordion': 0, 'airplanes': 1, 'anchor': 2, 'ant': 3, 'bass': 4, 'beaver': 5, 'binocular': 6, 'bonsai': 7, 'buddha': 8, 'butterfly': 9}


In [143]:
seed = 7
np.random.seed(seed)
X_train, X_test, y_train, y_test = train_test_split(imgs, labels, test_size = 0.2) # Change the proportion of training and test
X_train = np.stack(X_train, axis=0)
y_train = np.stack(y_train, axis=0)
X_test = np.stack(X_test, axis=0)
y_test = np.stack(y_test, axis=0)
# print ("Num train_imgs: %d" % (len(X_train))) # 8229 when test_size = 0.1
# print ("Num test_imgs: %d" % (len(X_test))) # 915 when test_size = 0.1
# # one hot encode outputs - change the single value into a vector with (number of categories) dimensions
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
num_classes= y_test.shape[1]



data = {}
data['categories'] = categories
data['X_train'] = X_train
data['y_train'] = y_train
data['X_test'] = X_test
data['y_test'] = y_test


In [144]:
print(len(X_train))

48


In [145]:
warnings.filterwarnings('ignore')

# define model
vgg16 = VGG16(weights='imagenet')
#vgg16.summary()

# don't train existing weights
for layer in vgg16.layers:
    layer.trainable = False

# Extracrting last leayer from the teh CNN model before the rediction layer
model = Model(inputs=vgg16.input, outputs=vgg16.get_layer('fc2').output)
temp_labels = []
temp_features = []


for index in range(0, len(X_train)):
    x = X_train[index]
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    # Extracting features
    block_pool_features = model.predict(x)
    temp_labels.append(y_train[index])
    # convert the probabilities to class labels
    label = decode_predictions(block_pool_features)
    # retrieve the most likely result, e.g. highest probability
    label = label[0][0]
    # print the classification
    print('%s (%.2f%%)' % (label[1], label[2]*100))

    flat_pool_features = block_pool_features.flatten()
    temp_features.append(flat_pool_features)


# Each image can have multiple labels assigned to it. Assigning identical labels to the new features.
#print(temp_features)
#print(temp_labels)

ValueError: `decode_predictions` expects a batch of predictions (i.e. a 2D array of shape (samples, 1000)). Found array with shape: (1, 4096)

In [148]:
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.applications.vgg16 import decode_predictions
from keras.applications.vgg16 import VGG16
# load the model
model = VGG16()
# load an image from file
#image = load_img('mug.jpg', target_size=(224, 224))
# convert the image pixels to a numpy array
#image = img_to_array(image)
# reshape data for the model
img = img.reshape((1, X_train[0][0], X_train[0][1], X_train[0][2]))
# prepare the image for the VGG model
img = preprocess_input(img)
# predict the probability across all output classes
yhat = model.predict(img)
# convert the probabilities to class labels
label = decode_predictions(yhat)
# retrieve the most likely result, e.g. highest probability
label = label[0][0]
# print the classification
print('%s (%.2f%%)' % (label[1], label[2]*100))

TypeError: only integer scalar arrays can be converted to a scalar index

In [151]:
img = load_img('/Users/utkarsh/Desktop/study/iitj/sem2/ml2/assignment2/101_ObjectCategories/bass/image_0001.jpg', target_size=(224, 224))
# convert the image pixels to a numpy array
img = img_to_array(img)
img = img.reshape((1, img.shape[0], img.shape[1], img.shape[2]))
img = preprocess_input(img)
# predict the probability across all output classes
yhat = model.predict(img)
# convert the probabilities to class labels
label = decode_predictions(yhat)
# retrieve the most likely result, e.g. highest probability
label = label[0][0]
# print the classification
print('%s (%.2f%%)' % (label[1], label[2]*100))



array([[[[ 40.,  47., 117.],
         [ 39.,  46., 116.],
         [ 38.,  45., 115.],
         ...,
         [ 37.,  45., 110.],
         [ 38.,  46., 111.],
         [ 38.,  46., 111.]],

        [[ 40.,  47., 117.],
         [ 39.,  46., 116.],
         [ 38.,  45., 115.],
         ...,
         [ 36.,  44., 109.],
         [ 36.,  44., 109.],
         [ 36.,  44., 109.]],

        [[ 40.,  47., 117.],
         [ 39.,  46., 116.],
         [ 38.,  45., 115.],
         ...,
         [ 36.,  44., 109.],
         [ 36.,  44., 109.],
         [ 36.,  44., 109.]],

        ...,

        [[ 23.,  34.,  98.],
         [ 33.,  44., 108.],
         [ 30.,  41., 105.],
         ...,
         [ 30.,  43.,  98.],
         [ 29.,  42.,  97.],
         [ 24.,  37.,  92.]],

        [[ 23.,  34.,  98.],
         [ 33.,  44., 108.],
         [ 30.,  41., 105.],
         ...,
         [ 30.,  43.,  98.],
         [ 29.,  42.,  97.],
         [ 24.,  37.,  92.]],

        [[ 23.,  34.,  98.],
       

In [153]:
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.applications.vgg16 import decode_predictions
from tensorflow.keras.applications.vgg16 import VGG16

img = load_img('/Users/utkarsh/Desktop/study/iitj/sem2/ml2/assignment2/101_ObjectCategories/bass/image_0001.jpg', target_size=(224, 224))
# convert the image pixels to a numpy array
img = img_to_array(img)
img = img.reshape((1, img.shape[0], img.shape[1], img.shape[2]))
img = preprocess_input(img)
# predict the probability across all output classes
yhat = model.predict(img)
# convert the probabilities to class labels
label = decode_predictions(yhat)
# retrieve the most likely result, e.g. highest probability
label = label[0][0]
# print the classification
print('%s (%.2f%%)' % (label[1], label[2]*100))

goldfish (86.72%)
